In [1]:
import numpy as np
import math
import pickle
from openpyxl import Workbook
from openpyxl import load_workbook
import mojimoji
import numpy as np
import glob
import re
import MeCab
import types
import pandas as pd
from collections import Counter
%matplotlib inline

mc = MeCab.Tagger("-Ochasen")

In [2]:
dirpath = "data/excel/"

filename = "A_W_171116.xlsx"
A_W_171116_excel = pd.ExcelFile(dirpath + filename)
sheet_name = A_W_171116_excel.sheet_names[0]
A_W_171116_df = A_W_171116_excel.parse(sheet_name)

filename = "A_W_171128.xlsx"
A_W_171128_excel = pd.ExcelFile(dirpath + filename)
sheet_name = A_W_171128_excel.sheet_names[0]
A_W_171128_df = A_W_171128_excel.parse(sheet_name)

filename = "171116.xlsx"
S_H_171116_excel = pd.ExcelFile(dirpath + filename)
sheet_name = S_H_171116_excel.sheet_names[0]
S_H_171116_df = S_H_171116_excel.parse(sheet_name)

filename = "2015斉藤.xlsx"
S_2015_excel = pd.ExcelFile(dirpath + filename)
sheet_name = S_2015_excel.sheet_names[0]
S_2015_df = S_2015_excel.parse(sheet_name)

## 必要なデータ
- id
    - A_W_YYMMDDのNo.(0列目)
    - SH_171116のid(0列目)
    - 斉藤2015の発言番号(0列目)
- reply
    - A_W_YYMMDDの返信元(1列目)
    - SH_171116のreply(3列目)
    - 斉藤2015の返信元(1列目)
- group
    - A_W_YYMMDDのgid(6列目)
    - SH_171116のgid(5列目)
    - 斉藤2015のグループ番号(6列目)
- who
    - A_W_YYMMDDのニックネーム(8列目)
    - SH_171116のcname(6列目)
    - 斉藤2015のニックネーム(8列目)
- body
    - A_W_YYMMDDの発言内容(11列目)
    - SH_171116のbody(8列目)
    - 斉藤2015の発言内容(11列目)
- epistemic_a
    - A_W_YYMMDDのEpistemic(A)(12列目)
    - Epistemic(H)(9列目)
- epistemic_b
    - A_W_YYMMDDのWatanabe(13列目)
    - Epistemic(S)(10列目)
- social_table_a
    - A_W_YYMMDDのSocial(A)(27列目)
    - SH_171116のSocial(H)(21列目)
- social_table_b
    - A_W_YYMMDDのWatanabe.3(28列目)
    - SH_171116のSocial(S)(22列目)
    

## 各dfで必要な列
- A_W_YYMMDD[0, 1, 6, 8, 11, 12, 13, 27, 28]
- SH_171116[0, 3, 5, 6, 8, 9, 10, 21, 22]
- 斉藤2015[0, 1, 6, 8, 11]

In [3]:
A_W_YYMMDD_use = [0, 1, 6, 8, 11, 12, 13, 27, 28]
S_H_171116_use = [0, 3, 5, 6, 8, 9, 10, 21, 22]
S_2015_use = [0, 1, 6, 8, 11]

In [4]:
# A_W_YYMMDDの列名の統合
A_W_171116_df.columns = A_W_171128_df.columns

delete_columns_list = []
for i, column in enumerate(A_W_171116_df):
    if not i in A_W_YYMMDD_use:
        delete_columns_list.append(column)
        
A_W_171116_df = A_W_171116_df.drop(delete_columns_list, axis=1)
A_W_171128_df = A_W_171128_df.drop(delete_columns_list, axis=1)

delete_columns_list = []
for i, column in enumerate(S_H_171116_df):
    if not i in S_H_171116_use:
        delete_columns_list.append(column)
S_H_171116_df = S_H_171116_df.drop(delete_columns_list, axis=1)

delete_columns_list = []
for i, column in enumerate(S_2015_df):
    if not i in S_2015_use:
        delete_columns_list.append(column)
S_2015_df = S_2015_df.drop(delete_columns_list, axis=1)

In [5]:
print(not False in S_H_171116_df['id'].values == S_2015_df['発言番号'].values)
print(not False in S_H_171116_df['cname'].values == S_2015_df['ニックネーム'].values)
print(not False in S_H_171116_df['body'].values == S_2015_df['発言内容'].values)

True
True
True


In [6]:
S_H_171116_df["reply"] = S_2015_df['返信元']
S_H_171116_df["gid"] = S_2015_df['グループ番号']
concat_border = 299
A_W_171116_df = A_W_171116_df[0:concat_border]
A_W_171128_df = A_W_171128_df[concat_border:]
A_W_df = pd.concat([A_W_171116_df, A_W_171128_df])

In [7]:
# 要素含まれるNanの数を確認
A_W_df.isnull().sum()

No.                3
返信元                3
gid                3
ニックネーム             3
発言内容               3
Epistemic(A)       3
Watanabe           3
Social(A)       2881
Watanabe.3      2746
dtype: int64

# 重要！！

In [8]:
# A_W_dfの人間が付けたラベル列のNanをBlankに変換
A_W_df = A_W_df.fillna('Blank')
# 要素にNanが含まれる列を再確認
A_W_df.isnull().any()

No.             False
返信元             False
gid             False
ニックネーム          False
発言内容            False
Epistemic(A)    False
Watanabe        False
Social(A)       False
Watanabe.3      False
dtype: bool

In [9]:
# 要素に含まれるNanの数を確認
S_H_171116_df.isnull().sum()

id                 0
reply              0
gid                0
cname              0
body               0
Epistemic(H)       1
Epistemic(S)       4
Social(H)       2832
Social(S)       2823
dtype: int64

# 重要！！

In [10]:
# S_H_171116_dfの人間が付けたラベル列のNanを取り除く
S_H_171116_df = S_H_171116_df.fillna('Blank')
# 要素にNanが含まれる列を再確認
S_H_171116_df.isnull().any()

id              False
reply           False
gid             False
cname           False
body            False
Epistemic(H)    False
Epistemic(S)    False
Social(H)       False
Social(S)       False
dtype: bool

# 重要！！

In [11]:
A_W_df = A_W_df.drop_duplicates(subset=['ニックネーム', '発言内容'])
S_H_171116_df = S_H_171116_df.drop_duplicates(subset=['cname', 'body'])

In [12]:
counter = Counter(A_W_df['Watanabe.3'])
print(counter)
A_W_df = A_W_df[A_W_df['Watanabe.3'] != 2723]
counter = Counter(A_W_df['Watanabe.3'])
print(counter)

Counter({'Blank': 2475, 'Externalization': 1192, 'Quick consensus building': 526, 'Elicitation': 194, 'Integrarion-oriented consensus building': 151, 'Conflict-oriented consensus building': 41, 'Summary': 23, 2723: 2})
Counter({'Blank': 2475, 'Externalization': 1192, 'Quick consensus building': 526, 'Elicitation': 194, 'Integrarion-oriented consensus building': 151, 'Conflict-oriented consensus building': 41, 'Summary': 23})


# 重要！！

In [13]:
# 表記ゆれの修正と確認
A_W_df['Social(A)'] = A_W_df['Social(A)'].str.lower()
counter = Counter(A_W_df['Social(A)'])
print(counter)
A_W_df['Watanabe.3'] = A_W_df['Watanabe.3'].str.lower()
counter = Counter(A_W_df['Watanabe.3'])
print(counter)

Counter({'blank': 2645, 'externalization': 856, 'quick consensus building': 563, 'elicitation': 231, 'integrarion-oriented consensus building': 111, 'integration-oriented consensus building': 99, 'summary': 56, 'conflict-oriented consensus building': 41})
Counter({'blank': 2475, 'externalization': 1192, 'quick consensus building': 526, 'elicitation': 194, 'integrarion-oriented consensus building': 151, 'conflict-oriented consensus building': 41, 'summary': 23})


# 重要！！

In [14]:
# 表記ゆれの確認
S_H_171116_df['Social(S)'] = S_H_171116_df['Social(S)'].str.lower()
counter = Counter(S_H_171116_df['Social(S)'])
print(counter)
S_H_171116_df['Social(H)'] = S_H_171116_df['Social(H)'].str.lower()
counter = Counter(S_H_171116_df['Social(H)'])
print(counter)

Counter({'blank': 2557, 'externalization': 834, 'quick consensus building': 690, 'elicitation': 268, 'integrarion-oriented consensus building': 233, 'conflict-oriented consensus building': 129, 'summary': 28})
Counter({'blank': 2601, 'externalization': 992, 'quick consensus building': 632, 'elicitation': 260, 'integrarion-oriented consensus building': 106, 'conflict-oriented consensus building': 80, 'integration-oriented consensus building': 42, 'summary': 26})


In [15]:
correct_label = 'integration-oriented consensus building'
incorrect_label = 'integrarion-oriented consensus building'

A_W_df['Social(A)'] = A_W_df['Social(A)'].where(A_W_df['Social(A)'] != incorrect_label, correct_label)
counter = Counter(A_W_df['Social(A)'])
print(counter)
A_W_df['Watanabe.3'] = A_W_df['Watanabe.3'].where(A_W_df['Watanabe.3'] != incorrect_label, correct_label)
counter = Counter(A_W_df['Watanabe.3'])
print(counter)

S_H_171116_df['Social(S)'] = S_H_171116_df['Social(S)'].where(S_H_171116_df['Social(S)'] != incorrect_label, correct_label)
counter = Counter(S_H_171116_df['Social(S)'])
print(counter)
S_H_171116_df['Social(H)'] = S_H_171116_df['Social(H)'].where(S_H_171116_df['Social(H)'] != incorrect_label, correct_label)
counter = Counter(S_H_171116_df['Social(H)'])
print(counter)

Counter({'blank': 2645, 'externalization': 856, 'quick consensus building': 563, 'elicitation': 231, 'integration-oriented consensus building': 210, 'summary': 56, 'conflict-oriented consensus building': 41})
Counter({'blank': 2475, 'externalization': 1192, 'quick consensus building': 526, 'elicitation': 194, 'integration-oriented consensus building': 151, 'conflict-oriented consensus building': 41, 'summary': 23})
Counter({'blank': 2557, 'externalization': 834, 'quick consensus building': 690, 'elicitation': 268, 'integration-oriented consensus building': 233, 'conflict-oriented consensus building': 129, 'summary': 28})
Counter({'blank': 2601, 'externalization': 992, 'quick consensus building': 632, 'elicitation': 260, 'integration-oriented consensus building': 148, 'conflict-oriented consensus building': 80, 'summary': 26})


In [16]:
# 表記ゆれの修正と確認
A_W_df['Epistemic(A)'] = A_W_df['Epistemic(A)'].str.lower()
counter = Counter(A_W_df['Epistemic(A)'])
print(counter)
A_W_df['Watanabe'] = A_W_df['Watanabe'].str.lower()
counter = Counter(A_W_df['Watanabe'])
print(counter)

Counter({'off task': 2392, 'on task': 2090, 'no sense': 119, 'blank': 1})
Counter({'off task': 2444, 'on task': 2146, 'no sense': 11, 'blank': 1})


In [17]:
# 表記ゆれの確認
S_H_171116_df['Epistemic(S)'] = S_H_171116_df['Epistemic(S)'].str.lower()
counter = Counter(S_H_171116_df['Epistemic(S)'])
print(counter)
S_H_171116_df['Epistemic(H)'] = S_H_171116_df['Epistemic(H)'].str.lower()
counter = Counter(S_H_171116_df['Epistemic(H)'])
print(counter)

Counter({'off task': 2546, 'on task': 2179, 'no sense': 10, 'blank': 4})
Counter({'off task': 2560, 'on task': 2165, 'no sense': 13, 'blank': 1})


In [18]:
A_W_df = A_W_df[A_W_df['Epistemic(A)'] != 'no sense']
A_W_df = A_W_df[A_W_df['Watanabe'] != 'no sense']
A_W_df = A_W_df[A_W_df['Epistemic(A)'] != 'blank']
A_W_df = A_W_df[A_W_df['Watanabe'] != 'blank']
print(Counter(A_W_df['Epistemic(A)']))
print(Counter(A_W_df['Watanabe']))

Counter({'off task': 2390, 'on task': 2089})
Counter({'off task': 2337, 'on task': 2142})


In [19]:
S_H_171116_df = S_H_171116_df[S_H_171116_df['Epistemic(H)'] != 'no sense']
S_H_171116_df = S_H_171116_df[S_H_171116_df['Epistemic(S)'] != 'no sense']
S_H_171116_df = S_H_171116_df[S_H_171116_df['Epistemic(H)'] != 'blank']
S_H_171116_df = S_H_171116_df[S_H_171116_df['Epistemic(S)'] != 'blank']
print(Counter(S_H_171116_df['Epistemic(H)']))
print(Counter(S_H_171116_df['Epistemic(S)']))

Counter({'off task': 2558, 'on task': 2158})
Counter({'off task': 2538, 'on task': 2178})


# 重要！！

In [20]:
A_W_match_df = A_W_df[A_W_df['Social(A)'] == A_W_df['Watanabe.3']].reset_index(drop=True)
A_W_match_df.head()

,No.,返信元,gid,ニックネーム,発言内容,Epistemic(A),Watanabe,Social(A),Watanabe.3
0,1,\N,1,まこぴす,よろしくお願いします！,off task,off task,blank,blank
1,31,\N,1,哲,よろしくお願いします,off task,off task,blank,blank
2,70,\N,1,仙波,名前なのが恥ずかしいです…\n\nよろしくお願いします！,off task,off task,blank,blank
3,119,\N,1,まこぴす,早速課題やっちゃいましょう！,off task,off task,blank,blank
4,302,\N,1,哲,同意です,on task,on task,quick consensus building,quick consensus building


# 重要！！

In [21]:
S_H_match_df = S_H_171116_df[S_H_171116_df['Social(H)'] == S_H_171116_df['Social(S)']].reset_index(drop=True)
S_H_match_df.head()

,id,reply,gid,cname,body,Epistemic(H),Epistemic(S),Social(H),Social(S)
0,1593,\N,1,世界のわたべ,よろしくお願いします。,off task,off task,blank,blank
1,1598,\N,1,ざきさん,よろしくです,off task,off task,blank,blank
2,1606,\N,1,あ,よろしくです,off task,off task,blank,blank
3,1659,\N,1,ざきさん,みなさんファイルをアップしましたか？,off task,off task,blank,blank
4,1683,\N,1,あ,今アップしました！,off task,off task,blank,blank


In [22]:
A_W_match_df.isnull().sum()

No.             0
返信元             0
gid             0
ニックネーム          0
発言内容            0
Epistemic(A)    0
Watanabe        0
Social(A)       0
Watanabe.3      0
dtype: int64

In [23]:
S_H_match_df.isnull().sum()

id              0
reply           0
gid             0
cname           0
body            0
Epistemic(H)    0
Epistemic(S)    0
Social(H)       0
Social(S)       0
dtype: int64

# 重要！！

In [24]:
header = ['id', 'reply', 'group_id', 'cname', 'body', 'Epistemic_A', 'Epistemic_B', 'Social_A', 'Social_B']
A_W_df.columns = header
S_H_171116_df.columns = header
A_W_match_df.columns = header
S_H_match_df.columns = header
All_df = pd.concat([A_W_df, S_H_171116_df], ignore_index=True)
match_df = pd.concat([A_W_match_df, S_H_match_df], ignore_index=True)

print(A_W_df.shape)
print(S_H_171116_df.shape)
print(All_df.shape)
print(match_df.shape)

(4479, 9)
(4716, 9)
(9195, 9)
(7183, 9)


In [25]:
# データの形式の定義
data_format = ['id', 'reply', 'group_id', 'cname', 'body', 'epistemic', 'label']
# 名前からデータのindexに変換する辞書
data_index = {column : i for i, column in enumerate(data_format)}
print(data_index)

id = 0
reply = 1
group_id = 2
cname = 3
body = 4
epistemic = 5
label = 6

df_index = {column: i for i, column in enumerate(All_df)}
print(df_index)

{'id': 0, 'cname': 3, 'label': 6, 'group_id': 2, 'epistemic': 5, 'body': 4, 'reply': 1}
{'id': 0, 'Social_A': 7, 'cname': 3, 'group_id': 2, 'Social_B': 8, 'Epistemic_A': 5, 'body': 4, 'reply': 1, 'Epistemic_B': 6}


# 重要！！

In [26]:
df_use = [0, 1, 2, 3, 4]

# All_df_index['ラベル名']を変更して取り出すラベルを分ける
# epistemic_position = df_index['Epistemic_A']
# epistemic_position = df_index['Epistemic_B']
epistemic_position = df_index['Epistemic_B']
df_use.append(epistemic_position)

# label_position = df_index['Social_A']
# label_position = df_index['Social_B']
label_position = df_index['Social_B']
df_use.append(label_position)

print(df_use)

delete_columns_list = []
for i, column in enumerate(All_df):
#for i, column in enumerate(match_df):
    if not i in df_use:
        delete_columns_list.append(column)
input_df = All_df.drop(delete_columns_list, axis=1)
#input_df = match_df.drop(delete_columns_list, axis=1)
# input_df = A_W_consulted_df.drop(delete_columns_list, axis=1)
input_df.columns = data_format

[0, 1, 2, 3, 4, 6, 8]


In [27]:
input_df

,id,reply,group_id,cname,body,epistemic,label
0,1,\N,1,まこぴす,よろしくお願いします！,off task,blank
1,31,\N,1,哲,よろしくお願いします,off task,blank
2,70,\N,1,仙波,名前なのが恥ずかしいです…\n\nよろしくお願いします！,off task,blank
3,119,\N,1,まこぴす,早速課題やっちゃいましょう！,off task,blank
4,163,\N,1,仙波,やっちゃいましょう\n\nmoodleはゴミです！,on task,blank
5,194,\N,1,まこぴす,使いにくいです(笑),on task,blank
6,302,\N,1,哲,同意です,on task,quick consensus building
7,309,\N,1,仙波,以前インタラクティブアート受講していたのですが、その時に課題が不具合で出せなくなっていた時期...,on task,externalization
8,385,\N,1,まこぴす,学習環境として必要最低限の機能は備えていると思うが、操作性の面ではPCの最低限の知識があるこ...,on task,externalization
9,426,\N,1,まこぴす,自分の意見です！(笑),on task,externalization


In [28]:
input_df = input_df[input_df['epistemic'] == 'on task'].reset_index(drop=True)
input_df

,id,reply,group_id,cname,body,epistemic,label
0,163,\N,1,仙波,やっちゃいましょう\n\nmoodleはゴミです！,on task,blank
1,194,\N,1,まこぴす,使いにくいです(笑),on task,blank
2,302,\N,1,哲,同意です,on task,quick consensus building
3,309,\N,1,仙波,以前インタラクティブアート受講していたのですが、その時に課題が不具合で出せなくなっていた時期...,on task,externalization
4,385,\N,1,まこぴす,学習環境として必要最低限の機能は備えていると思うが、操作性の面ではPCの最低限の知識があるこ...,on task,externalization
5,426,\N,1,まこぴす,自分の意見です！(笑),on task,externalization
6,500,\N,1,仙波,ほぼほぼ同意です,on task,quick consensus building
7,528,\N,1,仙波,他に何か考えてることなどありますか,on task,elicitation
8,693,\N,1,まこぴす,他に意見ありますかー？,on task,elicitation
9,715,\N,1,仙波,自分はないです,on task,externalization


In [29]:
counter = Counter(input_df['label'])
print(counter)

Counter({'externalization': 2016, 'quick consensus building': 1212, 'elicitation': 458, 'integration-oriented consensus building': 383, 'conflict-oriented consensus building': 170, 'summary': 50, 'blank': 31})


In [30]:
counter = Counter(input_df['label'])
ranked_data = counter.most_common()

ranked_data

[('externalization', 2016),
 ('quick consensus building', 1212),
 ('elicitation', 458),
 ('integration-oriented consensus building', 383),
 ('conflict-oriented consensus building', 170),
 ('summary', 50),
 ('blank', 31)]

In [31]:
input_data = input_df.values

In [32]:
def preprocess(xs, trim_citation=False):
    zs = [""] * len(xs)
    '''全角空白'''
    for i, l in enumerate(xs):
        xs[i] = re.sub('　', ' ', str(l))
    '''多重引用'''
    for i, l in enumerate(xs):
        xs[i] = re.sub('＆ｇｔ；', '＞', l)
    '''引用部分は除外'''
    count_refer = 0
    for i, l in enumerate(xs):
        sen = ""
        have_refer = 1
        for x in l.split('\n'):
            if len(x) == 0:
                continue
            elif x[0].encode('utf-8') == '＞'.encode('utf-8'):
                have_refer = 1
                x = x[1:]
                if len(x) > 0 and x[0].encode('utf-8') != '＞'.encode('utf-8') and \
                        x[0:2].encode('utf-8') != ' ＞'.encode('utf-8') and x != ' ':
                    if trim_citation:
                        zs[i] += x + '\n'
                    else:
                        sen = sen + "＞ " + x + '\n'
                continue
            else:
                sen = sen + x + '\n'
        xs[i] = sen.strip()
        count_refer += have_refer
    '''顔文字'''
    pat = re.compile('（[^ぁ-んァ-ン一-龠]+?）', re.U)
    for i, l in enumerate(xs):
        xs[i] = re.sub(pat, ' KAOMOJI ', l)
    '''日本語及び記号{？！ー〜。、} のみ有効'''
    pat = re.compile('[^＞ Ａ-Ｚａ-ｚ０-９ぁ-んァ-ン一-龠？！ー〜。、\n]+?', re.U)
    for i, l in enumerate(xs):
        xs[i] = re.sub(pat, '', l)
    '''記号繰り返し'''
    pat_1 = re.compile('[ｗ]{1,}', re.U)
    pat_2 = re.compile('[？]{1,}', re.U)
    pat_3 = re.compile('[！]{1,}', re.U)
    pat_4 = re.compile('[～]{1,}', re.U)
    pat_5 = re.compile('[ー]{1,}', re.U)
    pat_6 = re.compile('[。]{1,}', re.U)

    for i, l in enumerate(xs):
        l = re.sub(pat_1, 'ｗ', l)
        l = re.sub(pat_2, '？', l)
        l = re.sub(pat_3, '！', l)
        l = re.sub(pat_4, '～', l)
        l = re.sub(pat_5, 'ー', l)
        l = re.sub(pat_6, '。', l)
        xs[i] = l

    if trim_citation:
        return zs
    else:
        return None

In [33]:
def make_xs(orig, trim_citation=False, use_seq2seq=False):

    tmp = np.copy(orig)
    xs = tmp[:, body]
    zs = preprocess(xs, trim_citation=(trim_citation or use_seq2seq))

    if use_seq2seq:
        tmp_prev = np.roll(tmp, 1, axis=0)
        tmp_prev[0, 0] = ""
        ts = tmp[:, [1, 2]]
        xs_prev = tmp_prev[:, body]

        for i in range(len(tmp)):
            if tmp[i, group_id] != tmp_prev[i, group_id]:  # group が異なる
                xs_prev[i] = ""
        preprocess(xs_prev, trim_citation=True)

        for i in range(len(xs)):
            replay_to = tmp[i, 2]
            if zs[i] != "":
                xs_prev[i] = zs[i]
            elif replay_to != -1:
                a = xs[:i][tmp[:i, 1] == replay_to]
                if len(a) != 0:
                    xs_prev[i] = a[-1]

        return xs, xs_prev
    else:
        return xs

In [34]:
def get_doc(xs):
    doc = []
    for ll in xs:
        sen = []
        for l in ll.split('\n'):
            for w in mc.parse(l).split('\n'):
                mx = w.split('\t')
                if len(mx[0]) > 0:
                    sen.append(mx[0])
        doc.append(sen)
    return doc

In [35]:
xs, xs_pre = make_xs(input_data, trim_citation=True, use_seq2seq=True)
doc = get_doc(xs)
doc_pre = get_doc(xs_pre)
for i in doc[:10]:
    print(" ".join(i))

やっ ちゃ い ましょ う EOS は ゴミ です ！ EOS
使い にくい です 笑 EOS
同意 です EOS
以前 インタラクティブ アート 受講 し て い た の です が 、 その 時 に 課題 が 不具合 で 出せ なく なっ て い た 時期 が あり まし て EOS 本当に もろい です EOS
学習 環境 として 必要 最低限 の 機能 は 備え て いる と 思う が 、 操作 性 の 面 で は の 最低限 の 知識 が ある こと が 前提 な ので 誰 も が 利用 できる と は 言え ない 。 また から だ と 見 やすい レイアウト も スマフォ から だ と 見 にくい と 感じ た 。 EOS
自分 の 意見 です ！ 笑 EOS
ほぼ ほぼ 同意 です EOS
他 に 何 か 考え てる こと など あり ます か EOS
他 に 意見 あり ます か ー ？ EOS
自分 は ない です EOS


In [36]:
wd_set = Counter([x for sen in doc + doc_pre for x in sen])
wd_ary = np.array(list(wd_set.keys()))
wd_cnt = np.array(list(wd_set.values()))
wd_ary = wd_ary[np.argsort(wd_cnt)[::-1]]
wd_cnt.sort()
wd_cnt = wd_cnt[::-1]
wd_to_id = {wd: i for i, wd in enumerate(wd_ary)}
lb_to_id = {lb: i for i, lb in enumerate({x for x in input_data[:, label]})}
id_to_wd = {wd_to_id[wd]: wd for wd in wd_to_id.keys()}
id_to_lb = {lb_to_id[lb]: lb for lb in lb_to_id.keys()}
for i in range(10):
    print(list(wd_ary)[i], list(wd_cnt)[i])
    #print(list(wd_set.keys())[i], list(wd_set.values())[i])

EOS 15035
の 8295
が 5692
は 5549
に 5438
と 5351
、 5111
か 4823
です 4728
て 4651


# 重要！！

In [37]:
KIND = len(list(lb_to_id.keys()))
CUT_OFF = 2

print("words kinds:",
      len(wd_cnt), "words>=" + str(CUT_OFF) + ":", np.sum(wd_cnt >= CUT_OFF))
print("all words num:", np.sum(wd_cnt))
print("all words num:", np.sum(wd_cnt[wd_cnt >= CUT_OFF]))

other_id = np.sum(wd_cnt >= CUT_OFF)
wd_to_id.update({wd: other_id for wd in wd_ary[wd_cnt < CUT_OFF]})
print(other_id)
id_to_wd[other_id] = '[X]'

print(max(wd_to_id.values()))

with open('data/Social_B_id_to_wd.pickle', mode='wb') as f:
    pickle.dump(id_to_wd, f)
with open('data/Social_B_wd_to_id.pickle', mode='wb') as f:
    pickle.dump(wd_to_id, f)
with open('data/Social_B_wd_set.pickle', mode='wb') as f:
    pickle.dump(wd_set, f)
with open('data/Social_B_id_to_lb.pickle', mode='wb') as f:
    pickle.dump(id_to_lb, f)
pd.DataFrame(
    np.array(list(id_to_lb.items())).T, index=["ラベルid", "ラベルname"])

words kinds: 4660 words>=2: 4608
all words num: 221132
all words num: 221080
4608
4608


,0,1,2,3,4,5,6
ラベルid,0,1,2,3,4,5,6
ラベルname,blank,quick consensus building,elicitation,integration-oriented consensus building,conflict-oriented consensus building,summary,externalization


In [38]:
of = open("data/Social_B_edu_data.txt", "w")
for i, (sen, lb) in enumerate(zip(np.array(doc), input_data[:, label])):
    sen_str = "".join([str(wd_to_id[wd]) + " " for wd in sen])
    print_str = str(lb_to_id[lb]) + " " + str(lb_to_id[lb]) + " " + sen_str
    print(print_str, file=of)
of.close()

of = open("data/Social_B_edu_data_pre.txt", "w")
for i, sen in enumerate(np.array(doc_pre)):
    sen_str = "".join([str(wd_to_id[wd]) + " " for wd in sen])
    print(sen_str, file=of)
of.close()

print(max([len(sen) for sen in doc]))

print(KIND)

269
7


# train